Este notebook tiene como objetivo procesar y preparar los datos para el desarrollo de un sistema de recomendación de películas, se trabajará en la limpieza del dataset credits.

Descripción del archivo credits.csv

El archivo credits.csv contiene información sobre el reparto y el equipo de producción de diversas películas. Se compone de las siguientes columnas:

    id: Identificador único de la película.

    cast: Lista en formato JSON con detalles de los actores, incluyendo su nombre, personaje interpretado, y otros datos.

    crew: Lista en formato JSON con detalles del equipo de producción, como el director, guionista y otros miembros clave.

⚙️ Proceso a realizar:
1️⃣ Cargar y visualizar la información del dataset.
2️⃣ Separar el DataFrame en dos archivos: uno con cast y otro con crew.
3️⃣ Reducir el tamaño eliminando el 40% de los registros.
se eliminaran estos registros por limitaciones de memoria 
4️⃣ Exportar los nuevos archivos en un formato más eficiente (Parquet).


In [ ]:
import pandas as pd # type: ignore
import json
import os

In [2]:

# Cargar el archivo CSV
file_path = r"C:\Users\E B M\Documents\proyecto_mp\dataset_org\credits.csv"
credits_df = pd.read_csv(file_path)

In [3]:

#  Mostrar información general del DataFrame
print("\n Información general del DataFrame:")
print(credits_df.info())


 Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


In [4]:
#  Mostrar las primeras filas del DataFrame
print("\n Primeras 5 filas del DataFrame:")
print(credits_df.head())




 Primeras 5 filas del DataFrame:
                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862  


In [5]:
#  Verificar valores nulos
print("\n Valores nulos por columna:")
print(credits_df.isnull().sum())




 Valores nulos por columna:
cast    0
crew    0
id      0
dtype: int64


In [6]:
#  Tipos de datos de cada columna
print("\n Tipos de datos de cada columna:")
print(credits_df.dtypes)




 Tipos de datos de cada columna:
cast    object
crew    object
id       int64
dtype: object


In [7]:
#  Estadísticas generales (para columnas numéricas)
print("\n Estadísticas generales (columnas numéricas):")
print(credits_df.describe())




 Estadísticas generales (columnas numéricas):
                  id
count   45476.000000
mean   108345.997537
std    112443.796536
min         2.000000
25%     26443.250000
50%     60002.500000
75%    157302.000000
max    469172.000000


In [8]:
#  Visualizar algunas filas de las columnas 'cast' y 'crew'
print("\n Ejemplo de valores en la columna 'cast':")
print(credits_df['cast'].head(5))

print("\n Ejemplo de valores en la columna 'crew':")
print(credits_df['crew'].head(5))


 Ejemplo de valores en la columna 'cast':
0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

 Ejemplo de valores en la columna 'crew':
0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object


Vamos a crear dos DataFrames, uno para el reparto (cast) y otro para el equipo de producción (crew):

In [9]:
# Crear DataFrame para el reparto (cast)
cast_df = credits_df[['id', 'cast']].copy()

# Crear DataFrame para el equipo de producción (crew)
crew_df = credits_df[['id', 'crew']].copy()

# Mostrar información después de la separación
print("\n DataFrame de Cast:")
print(cast_df.info())

print("\n DataFrame de Crew:")
print(crew_df.info())



 DataFrame de Cast:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45476 non-null  int64 
 1   cast    45476 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.7+ KB
None

 DataFrame de Crew:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45476 non-null  int64 
 1   crew    45476 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.7+ KB
None


Reducir el tamaño el 40% de los registros Para optimizar la memoria

In [10]:
# Reducir el 40% de los registros de cada DataFrame
cast_df = cast_df.sample(frac=0.6, random_state=42).reset_index(drop=True)
crew_df = crew_df.sample(frac=0.6, random_state=42).reset_index(drop=True)

# Mostrar la cantidad de registros después de la reducción
print(f"\n🔹 Registros en cast después de la reducción: {cast_df.shape[0]}")
print(f"\n🔹 Registros en crew después de la reducción: {crew_df.shape[0]}")



🔹 Registros en cast después de la reducción: 27286

🔹 Registros en crew después de la reducción: 27286




Finalmente, guardaremos los datos en la carpeta dataset_limpio en formato Parquet, que es más eficiente en almacenamiento y velocidad.

In [14]:

# Ruta de exportación
export_path = r"C:\Users\E B M\Documents\proyecto_mp\dataset_limpio"
os.makedirs(export_path, exist_ok=True)

# Definir rutas de los archivos
cast_file = os.path.join(export_path, "cast.parquet")
crew_file = os.path.join(export_path, "crew.parquet")

# Guardar en formato Parquet
cast_df.to_parquet(cast_file, index=False)
crew_df.to_parquet(crew_file, index=False)

print(f"Archivos exportados en: {export_path}")
print(f" - Cast: {cast_file}")
print(f" - Crew: {crew_file}")


Archivos exportados en: C:\Users\E B M\Documents\proyecto_mp\dataset_limpio
 - Cast: C:\Users\E B M\Documents\proyecto_mp\dataset_limpio\cast.parquet
 - Crew: C:\Users\E B M\Documents\proyecto_mp\dataset_limpio\crew.parquet
